# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
I will say this is a classification problem. 
We want to classify students to "will pass" and "fail" and give intervention to "fail" group.  

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import display

# Read student data
student_data = pd.read_csv("G:/onedrive/document/study/Udaicity/project2/programs/student-data.csv")
print "Student data read successfully!"
display(student_data.head())

Student data read successfully!


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [6]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [5]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print y_all.head()

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [8]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [9]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split 

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

Logistic Regression:

General application: Logistic regression have wide usage in Underwriting models, we use it to calculte % of chance a customer turns "bad" in a certain inquiry(EX:Will have payment default in a loan). 

Logistic Regression is general way to predict possiblity to be in a class. Think about two class classification problem, logistic regression will predict what is probability a records can be in class A. It have a assumption of smooth decision boundary. 
Logistic Regression is better methods in dealing with high dimension probelem, high L1 and L2 regularization will help avoid overfitting. 
Logistic Regression is also easy interpretation. 
Weakeness of logistic if one predictor and almost explain variable it almost can not do predict well. 

In data, there is 48 features/only 300 training example, simple PCA(require large samples to derive stable solutions) can not handle those feasture very good on small training size so do other simpe methods like FA. Logistic regression (as a pretty matured regerssion algorithm) can train very fast with those small trainig sample size and generate pretty good result. 


Decision Tree with Random Forest:

General application: Random forest have good application in industry too, cause randome foreest can reduce variance with #number of trees increasing. ID Analytics is using random forest to generate risk score for each consumer inquiry. (Easy to implement is RF's biggest advantages).   

Random forest is a ensemble methods build upon decision tree. It is a general methods using for classification. It can automatic handle multiple features and interactions. Random forest is very flexible:  they can be used with large number of attributes, small or large datasets. Training random forest is very fast. But disadvantages of random forest is less accuracy compare to BOOST/GBM. It is also fast too train but much slower to create prediction once trained. Random forests also have been observed to overfit for some datasets with noisy classification/regression tasks.

Similar as lgistic regression, decision tree can handle those 48 features pretty good plus their interactions. Random forest can generate a good map of whole population with 300 training example. Prediction on small sample is still very quick.   

SVM:

General application: One of typical usage is classification of images. SVMs have good model generalization when parameter and kernel chosen properly. Related image classification, hand writting recognition can also use SVMS. 

Concept of SVM is to archive biggest margin. SVM with well-defined kernal can do very good job in model generalization. SVMs usually use in linear classification, with kernel trick it can also handle no-linear classification pretty good. SVM also have a regularization parameter to aviod overfitting. Disadvantage of SVMs is it relies on Kernal parameter choice and regularization terms, kernel methods is pretty sensitive to paramter choice.SVMs is also have limitation on training size and speed. Optimal SVM mutli-class classification's methods is still under research. 

SVM can perform very good on small training sample, and also we can adjust soft-margin parameter for accuray. 


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [11]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = LogisticRegression()
clf_B = RandomForestClassifier(max_depth=3,min_samples_split=20, n_estimators=200)
clf_C = SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train.iloc[np.arange(100)]
y_train_100 = y_train.iloc[np.arange(100)]

X_train_200 = X_train.iloc[np.arange(200)]
y_train_200 = y_train.iloc[np.arange(200)]

X_train_300 = X_train.iloc[np.arange(300)]
y_train_300 = y_train.iloc[np.arange(300)]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0130 seconds.
F1 score for training set: 0.8593.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7647.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8562.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7914.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8468.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.8060.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.4110 seconds


Made predictions in 0.0100 seconds.
F1 score for training set: 0.8153.
Made predictions in 0.0100 seconds.
F1 score for test set: 0.7785.
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.3080 seconds


Made predictions in 0.0110 seconds.
F1 score for training set: 0.8364.
Made predictions in 0.0080 seconds.
F1 score for test set: 0.7662.
Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.3050 seconds


Made predictions in 0.0130 seconds.
F1 score for training set: 0.8333.
Made predictions in 0.0090 seconds.
F1 score for test set: 0.7662.
Training a SVC using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained model in 0.0030 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8679.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7815.
Training a SVC using a training set size of 300. . .
Trained model in 0.0070 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8761.
Made predictions in 0.0020 seconds.


F1 score for test set: 0.7838.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0030 seconds|0.0000 seconds|0.8593|0.7647|
| 200               |0.0020 seconds|0.0000 seconds|0.8562|0.7914|
| 300               |0.0030 seconds|0.0000 seconds|0.8468|0.8060|

** Classifer 2 - Random Forest**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.4110 seconds|0.0100 seconds|0.8153|0.7785|
| 200               |0.3080 seconds|0.0080 seconds|0.8364|0.7662|
| 300               |0.3050 seconds|0.0090 seconds|0.8333|0.7662|

** Classifer 3 - SVM**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0010 seconds|0.0010 seconds|0.8777|0.7746|
| 200               |0.0030 seconds|0.0010 seconds|0.8679|0.7815|
| 300               |0.0070 seconds|0.0020 seconds|0.8761|0.7838|

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Based on above table, Logistic Regression performe best in 3 models. It have lowest training+prediction time and also get the best F1 score in test and train. 

available data: Logistic regression get over perform random forest and SVM across all 100/200/300 training sample. Lmitied resources and cost: Logistic Regression have same training time on 100/200 training size of SVMs but have lowest prediction time cost over all other models. For performance standpoint, Logistic regression over perform all other model based on test F1 socre. 


 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
Problem: We want to predict if a student will fail/pass. 

Logistic regression predict base on probability. so for a students with certain feature:

0-- this student will absolutely fail

1-- this student will absolutely pass

for example, if a student have romantic. we know this will make he have lower chance to pass. 
assume 100 students, 50 of them have romantic relationship, 20 of them not pass. So we can say if a students have romantic relatioship, there is 20/50= 40% chance a student will not pass.   
Still above those 50 have romantic relationship, 30 of them have internet and 15 of those 30 students failed. so if a students have romantic relationship and internet, 15/30 = 50% fail. 

Logistic regression algorithm combine all feature we have, based on each students's feature, it combine them together and give a %chance of this student will fail/pass. 

In training process, we definitely want to get best model.(smallest mistake we make). Mistake is measure if model predict yes, real lable is yes. then mistake = 0. else for than training example, mistake = 1. we want to minimize #mistakes we make in model. 

Model it self will take all features data(example: romantic or not/have internet or not) from training data after finish training, and combine all those feature together give out:

Example:

 romantic  have internet   chance of fail%

 yes         no                 15%

 no          yes                15%

 no          no                 5%

 yes         yes                50% 

After we done with data training, we want to know how it perform on general data (Model haven't look on yet).

We use part of test data which model never seen it before and that part of data comes from same data source, get a image how model will perform on furture general use. So we can know if our previous model done probable or not and determine if we need to adjust our model. 

   


 


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [127]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score
import math

# TODO: Create the parameters list you wish to tune
parameters = {'C': [x for x in np.arange(0.01, 2, 0.01)] ,'penalty': ['l1','l2']}

# TODO: Initialize the classifier
lgr = LogisticRegression(tol=0.00001,max_iter=1000, random_state=24)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes', greater_is_better=True)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=lgr, param_grid=parameters, cv=5, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print grid_obj.best_estimator_
# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

LogisticRegression(C=0.13, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=24, solver='liblinear', tol=1e-05,
          verbose=0, warm_start=False)
Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8309.
Made predictions in 0.0000 seconds.
Tuned model has a testing F1 score of 0.8000.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
best F1 for training is : 0.8309.

best F1 for testing is : 0.8000.

F1 score on test side is a little bit lower than the untuned model, or say almost eaqual to each other. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.